Start
|
|--- Extraversion (E) – Introversion (I)
|    |--- Do you feel more energized when surrounded by people? (E)
|    |--- Do you often find solitude more refreshing than social gatherings? (I)
|    |--- When faced with a problem, do you prefer discussing it with others? (E)
|    |--- Do you tend to process your thoughts internally before you speak? (I)
|    |--- At parties, do you initiate conversations with new people? (E)
|    |--- Do you prefer spending weekends quietly at home rather than going out? (I)
|
|--- Sensing (S) – Intuition (N)
|    |--- Do you focus more on the details and facts of your immediate surroundings? (S)
|    |--- Are you more interested in exploring abstract theories and future possibilities? (N)
|    |--- In learning something new, do you prefer hands-on experience over theory? (S)
|    |--- Do you often think about how actions today will affect the future? (N)
|    |--- When planning a vacation, do you prefer having a detailed itinerary? (S)
|    |--- Do you enjoy discussing symbolic or metaphorical interpretations of a story? (N)
|
|--- Thinking (T) – Feeling (F)
|    |--- When making decisions, do you prioritize logic over personal considerations? (T)
|    |--- Are your decisions often influenced by how they will affect others emotionally? (F)
|    |--- In arguments, do you focus more on being rational than on people's feelings? (T)
|    |--- Do you strive to maintain harmony in group settings, even if it means compromising? (F)
|    |--- Do you often rely on objective criteria to assess situations? (T)
|    |--- When a friend is upset, is your first instinct to offer emotional support rather than solutions? (F)
|
|--- Judging (J) – Perceiving (P)
     |--- Do you prefer to have a clear plan and dislike unexpected changes? (J)
     |--- Are you comfortable adapting to new situations as they happen? (P)
     |--- Do you set and stick to deadlines easily? (J)
     |--- Do you enjoy being spontaneous and keeping your options open? (P)
     |--- Do you find satisfaction in completing tasks and finalizing decisions? (J)
     |--- Do you prefer exploring various options before making a decision? (P)


In [4]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.fetchers import LinkContentFetcher
from haystack.components.converters import HTMLToDocument
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter

from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.generators import GPTGenerator

from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_KEY')


In [ ]:
mbti_urls = ["https://www.verywellmind.com/istj-introversion-sensing-thinking-judgment-2795992", 
             "https://www.verywellmind.com/istp-introverted-sensing-thinking-perceiving-2795993",
             "https://www.verywellmind.com/isfj-introverted-sensing-feeling-judging-2795990",
             'https://www.verywellmind.com/isfp-introverted-sensing-feeling-perceiving-2795991',
             'https://www.verywellmind.com/infj-introverted-intuitive-feeling-judging-2795978',
             'https://www.verywellmind.com/infp-a-profile-of-the-idealist-personality-type-2795987',
             'https://www.verywellmind.com/intj-introverted-intuitive-thinking-judging-2795988',
             'https://www.verywellmind.com/intp-introverted-intuitive-thinking-perceiving-2795989',
             'https://www.verywellmind.com/estp-extraverted-sensing-thinking-perceiving-2795986',
             'https://www.verywellmind.com/estj-extraverted-sensing-thinking-judging-2795985',
             'https://www.verywellmind.com/esfp-extraverted-sensing-feeling-perceiving-2795984',
             'https://www.verywellmind.com/esfj-extraverted-sensing-feeling-judging-2795983',
             'https://www.verywellmind.com/enfp-an-overview-of-the-champion-personality-type-2795980',
             'https://www.verywellmind.com/enfj-extraverted-intuitive-feeling-judging-2795979',
             'https://www.verywellmind.com/the-entp-personality-type-and-characteristics-2795982',
             'https://www.verywellmind.com/entj-personality-type-2795981']


In [2]:
questions = [
    {"content": "Do you feel more energized when surrounded by people?", "trait": "E"},
    {"content": "Do you often find solitude more refreshing than social gatherings?", "trait": "I"},
    {"content": "When faced with a problem, do you prefer discussing it with others?", "trait": "E"},
    {"content": "Do you tend to process your thoughts internally before you speak?", "trait": "I"},
    {"content": "At parties, do you initiate conversations with new people?", "trait": "E"},
    {"content": "Do you prefer spending weekends quietly at home rather than going out?", "trait": "I"},
    {"content": "Do you focus more on the details and facts of your immediate surroundings?", "trait": "S"},
    {"content": "Are you more interested in exploring abstract theories and future possibilities?", "trait": "N"},
    {"content": "In learning something new, do you prefer hands-on experience over theory?", "trait": "S"},
    {"content": "Do you often think about how actions today will affect the future?", "trait": "N"},
    {"content": "When planning a vacation, do you prefer having a detailed itinerary?", "trait": "S"},
    {"content": "Do you enjoy discussing symbolic or metaphorical interpretations of a story?", "trait": "N"},
    {"content": "When making decisions, do you prioritize logic over personal considerations?", "trait": "T"},
    {"content": "Are your decisions often influenced by how they will affect others emotionally?", "trait": "F"},
    {"content": "In arguments, do you focus more on being rational than on people's feelings?", "trait": "T"},
    {"content": "Do you strive to maintain harmony in group settings, even if it means compromising?", "trait": "F"},
    {"content": "Do you often rely on objective criteria to assess situations?", "trait": "T"},
    {"content": "When a friend is upset, is your first instinct to offer emotional support rather than solutions?", "trait": "F"},
    {"content": "Do you prefer to have a clear plan and dislike unexpected changes?", "trait": "J"},
    {"content": "Are you comfortable adapting to new situations as they happen?", "trait": "P"},
    {"content": "Do you set and stick to deadlines easily?", "trait": "J"},
    {"content": "Do you enjoy being spontaneous and keeping your options open?", "trait": "P"},
    {"content": "Do you find satisfaction in completing tasks and finalizing decisions?", "trait": "J"},
    {"content": "Do you prefer exploring various options before making a decision?", "trait": "P"},
]

# Example usage
responses = ['No', 'Yes', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 
             'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No']

for i, question in enumerate(questions):
    question['meta'] = {
        'trait': question['trait'],
        'answer': responses[i]
    }


In [3]:
from haystack import Document

documents = [Document(content=question["content"], meta=question["meta"]) for question in questions]

In [6]:
prompt_template = """
Determine the personality of someone using the Myers-Briggs Type Indicator (MBTI) test. In this test,
a user has answeres a series of questions using "Yes" and "No" responses. The questions are
labelled according to the trait, where the traits are:
E = Extraversion
I = Introversion
S = Sensing
N = Intuition
T = Thinking
F = Feeling
J = Judging
P = Perceiving
If the documents do not contain the answer to the question, say that ‘Answer is unknown.’
Context:
{% for doc in documents %}
    Question: {{ doc.content }} Response: {{ doc.meta['answer'] }} Personality trait: {{doc.meta['trait']}} \n
{% endfor %};
Question: {{query}}
\n
"""
splitter = DocumentSplitter(split_length=100, split_overlap=5)

prompt_builder = PromptBuilder(prompt_template)
############################################
query_embedder = SentenceTransformersTextEmbedder()
llm = GPTGenerator(api_key=api_key)

In [7]:
pipeline = Pipeline()
pipeline.add_component("splitter", splitter)
pipeline.add_component(name="prompt_builder", instance=prompt_builder)
pipeline.add_component(name="llm", instance=llm)
pipeline.connect("splitter.documents", "prompt_builder.documents")
pipeline.connect("prompt_builder", "llm")

In [9]:
query="Based on the responses, what is this user's Myers-Briggs personality type?"
pipeline.run(data={'splitter': {'documents': documents}, "prompt_builder": {"query": query}})

{'llm': {'replies': ["Based on the responses provided, this user's Myers-Briggs personality type is ISTJ (Introversion, Sensing, Thinking, Judging)."],
  'meta': [{'model': 'gpt-3.5-turbo-0613',
    'index': 0,
    'finish_reason': 'stop',
    'usage': {'completion_tokens': 30,
     'prompt_tokens': 716,
     'total_tokens': 746}}]}}

In [ ]:
query = "How do I use the openai embedder?"
result = pipeline.run(data={"query_embedder": {"text": query}, "prompt_builder": {"query": query}})
print(result['llm']['replies'][0])